In [106]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.model_selection import KFold
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

# collect data
# ../Data/DataForClassification/d2v/
fileDir = "../Data/DataForClassification/d2v/"
fileList = os.listdir(fileDir)
print(fileList)

# # auto method that go through all the file in directory
# for file in fileList:
#     if not file.startswith('.'):
#         if file.endswith(".txt"):
#             file = file[:-4]

# hard code to read the file one by one
author0 = []
author1 = []
with open(fileDir+"david g lloyd0.txt", 'r', encoding = 'utf8') as f:
    for line in f:
        read_data = line.strip().split("\t")
        author0.append(read_data[1].split(" "))

with open(fileDir+"david g lloyd1.txt", 'r', encoding = 'utf8') as f:
    for line in f:
        read_data = line.strip().split("\t")
        author1.append(read_data[1].split(" "))
print(author0[0])
print(author1[0])

['chung-may yang1.txt', 'wei lu0.txt', 'yong wang1.txt', 'david g lloyd0.txt', 'wei lu1.txt', 'feng liu1.txt', 'david g lloyd1.txt', 'jeong hwan kim1.txt', 'chung-may yang0.txt', 'michael wagner0.txt', 'feng liu0.txt', 'hao song1.txt', 'hao song0.txt', 'kevin m. ryan0.txt', 'michael wagner1.txt', 'lei wang0.txt', 'jeong hwan kim0.txt', 'yong wang0.txt', 'lei wang1.txt', 'kevin m. ryan1.txt']
['0.36922291', '-0.19795579', '0.49601007', '0.56937379', '0.23814982', '-0.20741679', '0.02322432', '-0.09128307', '0.10833433', '-0.44993171', '-0.17381026', '0.21698874', '0.15634133', '-0.21289150', '-0.22035176', '-0.66691130', '-0.08266206', '0.52953374', '-0.52283341', '1.01679730', '0.17747700', '0.28360513', '-0.64240116', '0.69723111', '0.36909986', '0.06061623', '-0.27225286', '0.26881346', '0.57831347', '0.45311663', '0.33036721', '0.13136637', '-0.20699488', '0.03498034', '-0.00601116', '0.77119726', '0.53693563', '-0.39640459', '0.45612192', '-0.17980313', '0.05839377', '0.11145472', 

In [107]:
# size of each class
print(len(author0))
print(len(author1))

50
104


In [108]:
# number of features (dimension)
print(len(author0[0]))
print(len(author1[1]))

100
100


In [109]:
# reconstract data so that we can feed it to svm
import pandas as pd
classOne = pd.DataFrame(author0)
classOne["label"] = 0
#print(classOne[:2:])
classTwo = pd.DataFrame(author1)
classTwo["label"] = 1
#print(classTwo[:2:])
# combine data from different class get all data
combinedData = pd.concat([classOne, classTwo])
print(combinedData[:5])
combinedData = combinedData.sample(frac=1).reset_index(drop=True)
# split data and label
data = combinedData.drop('label', axis=1)
label = combinedData['label']

             0            1            2           3           4            5  \
0   0.36922291  -0.19795579   0.49601007  0.56937379  0.23814982  -0.20741679   
1   0.44019809   0.56984979  -0.05801425  0.10724957  0.81136632   0.14514671   
2   0.14738785  -0.16762426   0.44191498  0.77328289  0.30972996   0.09072363   
3   0.46882892  -0.53388768   0.41525257  0.30116102  0.66694850   0.48965639   
4  -0.04815168   0.02721202   0.07970838  0.05371442  0.46587369  -0.13458404   

            6            7           8            9  ...            91  \
0  0.02322432  -0.09128307  0.10833433  -0.44993171  ...    0.40918887   
1  0.06338627  -0.13445675  0.78733879  -0.02049698  ...    0.34250227   
2  0.14121903  -0.15820582  0.04141365  -0.38626724  ...    0.09026022   
3  0.23758464   0.15237521  0.54638553   0.25229490  ...   -0.45478535   
4  0.23646261   0.04194793  0.12056416  -0.01970780  ...    0.33367929   

            92          93          94           95           96    

In [110]:
# cross validation
def k_fold_cv(data, label, classifier, clfname):
    kf = KFold(n_splits=10, shuffle=True)
    # create lists to collect statistic
    tp = []
    fp = []
    tn = []
    fn = []
    roundf1 = []
    for train_index, test_index in kf.split(data):
        # print("TRAIN:", train_index, " \n TEST:", test_index)
        # split train and test
        data_train, data_test = data.iloc[train_index], data.iloc[test_index]
        label_train, test_true_label = label.iloc[train_index], label.iloc[test_index]
        # fit data to svm
        classifier.fit(data_train, label_train)
        # get predicted label
        label_pred = classifier.predict(data_test)
        # find round confusion matrix
        round_tn, round_fp, round_fn, round_tp = metrics.confusion_matrix(test_true_label, label_pred).ravel()
        # add data data to array
        tp.append(round_tp)
        fp.append(round_fp)
        fn.append(round_fn)
        tn.append(round_tn)
        roundf1.append(f1_score(test_true_label, label_pred,average='micro'))
        # print("True positive: {tp}, False positive: {fp}, False negative: {fn}, True negative: {tn}"
        # .format(tp=round_tp, fp=round_fp, fn=round_fn, tn=round_tn))

    print("Classifier: {name}\nTrue positive: {tp}, False positive: {fp}, False negative: {fn}, True negative: {tn}"
          .format(name=clfname, tp=np.sum(tp), fp=np.sum(fp), fn=np.sum(fn), tn=np.sum(tn)))
    f1 = np.average(roundf1)
    ppv, npv, specificity, sensitivity, accuracy = calculate_important_value(np.sum(tp), np.sum(tn),
                                                                             np.sum(fp), np.sum(fn), len(data),f1)
    return ppv, npv, specificity, sensitivity, accuracy, f1

In [111]:
# calculate ppv,npv,specificity,sensitivity, and accuracy
def calculate_important_value(tp, tn, fp, fn, sample_length,f1):
    # 1. Positive predicted value (PPV) or precision aka hit rate = True positive/ )True positive + False positive)
    ppv = (tp / (tp + fp))
    # 2. Negative predicted value (NPV) = True negative / (True negative + False negative)
    npv = (tn / (tn + fn))
    # 3. Specificity = (1 - False positive)
    specificity = (tn / (tn + fp))
    # 4. Sensitivity = True positive
    sensitivity = (tp / (tp + fn))
    # 5. Accuracy = (True positive + True negative) / Total number of sample
    accuracy = (tp + tn) / sample_length
    print('PPV: ', ppv)
    print('NPV: ', npv)
    print('Specificity: ', specificity)
    print('Sensitivity: ', sensitivity)
    print('Accuracy: ', accuracy)
    print('F1: ', f1)
    return ppv, npv, specificity, sensitivity, accuracy

In [112]:
# # create linear SVM model
linear_svc = svm.SVC(kernel='linear', class_weight='balanced')
print(linear_svc)

# fit model and do 10-fold cv
k_fold_cv(data, label, linear_svc, "SVM linear")

print()
# create rbf SVM model
rbf_svc = svm.SVC(kernel='rbf', C=10)
print(rbf_svc)

# fit model and do 10-fold cv
k_fold_cv(data, label, rbf_svc, "SVM rbf")

SVC(C=1.0, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
Classifier: SVM linear
True positive: 102, False positive: 2, False negative: 2, True negative: 48
PPV:  0.9807692307692307
NPV:  0.96
Specificity:  0.96
Sensitivity:  0.9807692307692307
Accuracy:  0.974025974025974
F1:  0.9737500000000001

SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
Classifier: SVM rbf
True positive: 102, False positive: 1, False negative: 2, True negative: 49
PPV:  0.9902912621359223
NPV:  0.9607843137254902
Specificity:  0.98
Sensitivity:  0.9807692307692307
Accuracy:  0.9805194805194806
F1:  0.9808333333333333


(0.9902912621359223,
 0.9607843137254902,
 0.98,
 0.9807692307692307,
 0.9805194805194806,
 0.9808333333333333)